In [1]:
from bokeh.sampledata.sprint import sprint
sprint[:10]

,Name,Country,Medal,Time,Year
0,Usain Bolt,JAM,GOLD,9.63,2012
1,Yohan Blake,JAM,SILVER,9.75,2012
2,Justin Gatlin,USA,BRONZE,9.79,2012
3,Usain Bolt,JAM,GOLD,9.69,2008
4,Richard Thompson,TRI,SILVER,9.89,2008
5,Walter Dix,USA,BRONZE,9.91,2008
6,Justin Gatlin,USA,GOLD,9.85,2004
7,Francis Obikwelu,POR,SILVER,9.86,2004
8,Maurice Greene,USA,BRONZE,9.87,2004
9,Maurice Greene,USA,GOLD,9.87,2000


In [3]:
from bokeh.io import output_notebook, show
from bokeh.models.glyphs import Circle, Text
from bokeh.models import ColumnDataSource, Range1d, DataRange1d, Plot

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

gold_fill   = "#efcf6d"
gold_line   = "#c8a850"
silver_fill = "#cccccc"
silver_line = "#b0b0b1"
bronze_fill = "#c59e8a"
bronze_line = "#98715d"

fill_color = { "gold": gold_fill, "silver": silver_fill, "bronze": bronze_fill }
line_color = { "gold": gold_line, "silver": silver_line, "bronze": bronze_line }

def selected_name(name, medal, year):
    return name if medal == "gold" and year in [1988, 1968, 1936, 1896] else None

t0 = sprint.Time[0]

sprint["Abbrev"]       = sprint.Country
sprint["Country"]      = sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])
sprint["Medal"]        = sprint.Medal.map(lambda medal: medal.lower())
sprint["Speed"]        = 100.0/sprint.Time
sprint["MetersBack"]   = 100.0*(1.0 - t0/sprint.Time)
sprint["MedalFill"]    = sprint.Medal.map(lambda medal: fill_color[medal])
sprint["MedalLine"]    = sprint.Medal.map(lambda medal: line_color[medal])
sprint["SelectedName"] = sprint[["Name", "Medal", "Year"]].apply(tuple, axis=1).map(lambda args: selected_name(*args))

source = ColumnDataSource(sprint)

In [6]:
plot_options = dict(plot_width = 800, plot_height = 480, toolbar_location = None,
                   outline_line_color = None, title = "Usain Bolt vs. 116 years of Olympic sprinters")

In [7]:
radius = dict(value = 5, units = "screen")

In [8]:
medal_glyph = Circle(x="MetersBack", y="Year", radius=radius, fill_color="MedalFill", 
                     line_color="MedalLine", fill_alpha=0.5)

athlete_glyph = Text(x="MetersBack", y="Year", x_offset=10, text="SelectedName",
    text_align="left", text_baseline="middle", text_font_size="9pt")

no_olympics_glyph = Text(x=7.5, y=1942, text=["No Olympics in 1940 or 1944"],
    text_align="center", text_baseline="middle",
    text_font_size="9pt", text_font_style="italic", text_color="silver")

In [9]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

In [10]:
show(plot)

In [11]:
from bokeh.models import Grid, LinearAxis, SingleIntervalTicker

In [13]:
xdr = Range1d(start = sprint.MetersBack.max()+2, end = 0)

In [14]:
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

In [15]:
xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [16]:
show(plot)

In [17]:
from bokeh.models import HoverTool

In [18]:
tooltips = """
<div>
    <span style="font-size: 15px;">@Name</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@Abbrev)</span>
</div>
<div>
    <span style="font-size: 17px; font-weight: bold;">@Time{0.00}</span>&nbsp;
    <span style="font-size: 10px; color: #666;">@Year</span>
</div>
<div style="font-size: 11px; color: #666;">@{MetersBack}{0.00} meters behind</div>
"""

In [20]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
medal = plot.add_glyph(source, medal_glyph)  # we need this renderer to configure the hover tool
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")


In [21]:
hover = HoverTool(tooltips=tooltips, renderers=[medal])
plot.add_tools(hover)

In [22]:
show(plot)

## Bokeh Tutorial 07. Annotation

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [2]:
output_notebook()

Loading BokehJS ...

In [5]:
import numpy as np
from bokeh.models.annotations import Span

x = np.linspace(0, 20, 200)
y = np.sin(x)

p = figure(y_range = (-2, 2))
p.line(x, y)

upper = Span(location = 1, dimension = 'width', line_color = 'olive', line_width =  4)
p.add_layout(upper)

lower = Span(location = -1, dimension = 'width', line_color = 'firebrick', line_width = 4)
p.add_layout(lower)
show(p)

In [6]:
import numpy as np
from bokeh.models.annotations import BoxAnnotation

In [7]:
x = np.linspace(0, 20, 200)
y = np.sin(x)

p = figure(y_range = (-2, 2))
p.line(x, y)
# region that always fills the top fo the plot
upper = BoxAnnotation(bottom = 1, fill_alpha = 0.1, fill_color = 'olive')
p.add_layout(upper)
# region that always fills the bottom of the plot

lower = BoxAnnotation(top = -1, fill_alpha = 0.1, fill_color = 'firebrick')
p.add_layout(lower)
#  a finite region
center = BoxAnnotation(top = 0.6, bottom = -0.3, left = 7, right = 12, fill_alpha = 0.1, fill_color = 'navy')
p.add_layout(center)

In [8]:
show(p)

In [10]:
from bokeh.plotting import figure, output_notebook, show, vplot

In [11]:
output_notebook()

Loading BokehJS ...

In [12]:
from numpy import cos, linspace

In [13]:
x = linspace(-6, 6, 100)
y= cos(x)

In [14]:
p = figure(width = 500, height = 500)
p.circle(x, y, size = 7, color = "firebrick", alpha = 0.5)
show(p)

In [15]:
from bokeh.sampledata.autompg import autompg

In [16]:
from numpy import array

In [17]:
grouped = autompg.groupby("yr")
mpg = grouped["mpg"]
avg = mpg.mean()
std = mpg.std()
years = array(list(grouped.groups.keys()))
american = autompg[autompg["origin"]==1]
japanese = autompg[autompg["origin"]==3]

In [18]:
p = figure()

p.quad(left = years-0.4, right = years+0.4, bottom = avg-std, top = avg+std, fill_alpha = 0.4)
p.circle(x = japanese["yr"], y = japanese["mpg"], size = 8,
        alpha = 0.4, line_color = "red", fill_color = None, line_width = 2)
p.triangle(x = american["yr"], y = american["mpg"], size = 8, 
          alpha = 0.4, line_color = "blue", fill_color = None, line_width = 2)
show(p)

In [19]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import gridplot

In [22]:
source = ColumnDataSource(autompg.to_dict("list"))
source.add(autompg["yr"], name = "yr")

plot_config = dict(plot_width=300, plot_height=300, tools="pan,wheel_zoom,box_zoom,box_select,lasso_select")

p1 = figure(**plot_config, title="MPG by Year")
p1.circle("yr", "mpg", color="blue", source=source)

p2 = figure(**plot_config, title="HP vs. Displacement")
p2.circle("hp", "displ", color="green", source=source)

p3 = figure(**plot_config, title = "MPG Vs. Displacement")
p3.circle("mpg", "displ", size = "cyl", line_color = "red", fill_color = None, source = source)

p = gridplot([[p1, p2, p3]])

In [23]:
show(p)

In [24]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

In [25]:
counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

In [26]:
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

In [27]:
county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]
county_colors = [colors[int(rate/3)] for rate in county_rates]

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    color=county_colors,
    name=county_names,
    rate=county_rates,
))

In [28]:
output_notebook()

Loading BokehJS ...

In [30]:
plot_config = dict(plot_width=300, plot_height=300, tools="pan,wheel_zoom,box_zoom,box_select,lasso_select")

p1 = figure(**plot_config, title="Bokeh Taxes")

p.patches('x', 'y', source=source,
          fill_color='color', fill_alpha=0.7,
          line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

AttributeError: 'GridPlot' object has no attribute 'patches'